In [55]:
from bs4 import BeautifulSoup
import datetime
import numpy as np
import pandas as pd
import re
import requests
import codecs
import os

wdir = "C:/Users/MarekSedlacek/Downloads/W251_Project/SilkRoad2/"

In [168]:
#os.chdir(wdir+"SilkRoad2/"+str(year)+"-"+str(month)+"-"+str(day)+"/products")
os.chdir(wdir+"SilkRoad2/2014-06-02/items")

#Reads in the HTML file
data = open("0-5-grams-pure-6-apb-powder.html",'r').read()
soup = BeautifulSoup(data)

In [181]:
##WORKING CELL##
non_decimal = re.compile(r'[^\d.]+')

sidebar = soup.find("div",attrs={"class":"sidebar"})
Category = sidebar.find("a").get_text()


body = soup.find("div",attrs={"class":"body"})
title = body.find("h2").get_text()
price = float(non_decimal.sub("",body.find("div",attrs={"class":"price_big"}).get_text()))
price_dollar = price*658.79
vendor = body.find("a").get_text()
S1 = body.find_all("p")
if S1[0].get_text()=="":
    S1 = S1[1].get_text().replace("ships from: ","").replace("ships to: ","")
else: S1 = S1[0].get_text().replace("ships from: ","").replace("ships to: ","")
Ship_From = S1.splitlines()[1]
Ship_To = S1.splitlines()[2]

In [81]:
#ACTUAL CODE FOR PULLING PRODUCT DATA

#Ensure the current directory is correctly set
os.chdir(wdir)
#Build a Pandas data frame to hold the new data
col_names = ["Title","Category","Price","Price Dollar","Vendor","Ships From", "Ships To", "Date"]
df = pd.DataFrame(columns = col_names)

c = 0 #Counter
non_decimal = re.compile(r'[^\d.]+')

#Loops through the directories (scraped dates)
for X in next(os.walk(os.getcwd()))[1]:
    #Sets the working directory to the item folder for the current date
    os.chdir(os.getcwd()+"\\"+X+"\items")
    curr_date = datetime.datetime(*[int(item) for item in X.split('-')])
    print os.getcwd()
    
    
    for i in os.listdir(os.getcwd()): #Loops through all files in the current directory
        try:
            if i.endswith(".html"): #Only reads HTML files
                c+=1
                
                soup = BeautifulSoup(open(i,'r').read()) #Reads in the HTML file

                
                sidebar = soup.find("div",attrs={"class":"sidebar"}) #Identifies the Sidebar section of the HTML doc
                Category = sidebar.find("a").get_text() #Pulls Category from the Sidebar

                body = soup.find("div",attrs={"class":"body"} )#Identifies the Body section of the HTML doc
                title = body.find("h2").get_text() #Pulls product title
                price = float(non_decimal.sub("",body.find("div",attrs={"class":"price_big"}).get_text())) #Pulls product price
                price_dollar = price*658.79 #Converts bitcoin price to Dollars (price from 2014-06-02)
                vendor = body.find("a").get_text() #Pulls Vendor
                
                #Identifies Shipping To and From out of a block of text
                S1 = body.find_all("p")
                if S1[0].get_text()=="":
                    S1 = S1[1].get_text().replace("ships from: ","").replace("ships to: ","")
                else: S1 = S1[0].get_text().replace("ships from: ","").replace("ships to: ","")
                Ship_From = S1.splitlines()[1]
                Ship_To = S1.splitlines()[2]
                
                #Creates a numpy array with the scraped values
                new = np.array([title,Category,price,price_dollar,vendor,Ship_From,Ship_To,curr_date])
                #Appends array to data frame
                df = df.append(pd.DataFrame(new, index=col_names).transpose())
                
                #Dictionary, for when we migrate to Cassandra
                row = {"Date":curr_date,"Title":title,"Category":Category,"Price":price,"price_dollar":price_dollar,"Vendor":vendor,"Ships_From":Ship_From,"Ships_To":Ship_To}

                #if c%100==0: print c #Tracking metric

            else:
                continue

        except:
            print "File " + i + " has failed"
            
    os.chdir(wdir) #Reset current working directory
        

C:\Users\MarekSedlacek\Downloads\W251_Project\SilkRoad2\2014-06-02\items
100
200
300
File raise-your-flag-sr-branded-flags-look-great-when-flying-from-your-totem%3Fvendor_feedback_page=3.html has failed
File rolex-daydate-president-full-yellow-gold-aaa%3Fvendor_feedback_page=26.html has failed
400
File victoria-s-secret-bikini-replica-leopard-black-white-vs7-15.html has failed
File yeah-we-ran-out-of-drugs-how-about-some-stickers-3-5-x-6-sr-in-white%3Fvendor_feedback_page=48.html has failed
File yeah-we-ran-out-of-drugs-how-about-some-stickers-4-x3-warhol-snowden%3Fvendor_feedback_page=48.html has failed
C:\Users\MarekSedlacek\Downloads\W251_Project\SilkRoad2\2014-06-11\items
500
600
700
800
900
1000
1100
1200


In [82]:
#Remove duplicates from "Feedback" pages
df = df.drop_duplicates(subset=['Vendor',"Title","Date"],keep="first")

#Re-indexes the dataframe
df.index = range(df.shape[0])

df

,Title,Category,Price,Price Dollar,Vendor,Ships From,Ships To,Date
0,0.1 Grams (1 point) of DMT sourced from Brazil...,Alcohol,0.069872,46.031,DimitryTOO,Australia,Australia,2014-06-02 00:00:00
1,0.1g (One Point) Crystal Methamphetamine / ICE,Stimulants,0.100882,66.4601,Dekay,Australia,Australia,2014-06-02 00:00:00
2,0.25g 2-DPMP,Stimulants,0.0481,31.6878,PurityStandard,Germany,Undeclared,2014-06-02 00:00:00
3,0.5 gr. Interways Cocaine ** Disco Shit**,Cocaine,0.098823,65.1036,The Interways,Germany,Worldwide,2014-06-02 00:00:00
4,0.5 grams crystal meth (ice) high quality,Stimulants,0.08294,54.64,Scaptain,Canada,Worldwide,2014-06-02 00:00:00
5,0.5 Grams Pure 6-APB Powder,Stimulants,0.047312,31.1687,geoffreygiraffe,United States,United States & Canada,2014-06-02 00:00:00
6,1/2 Oz of shatter / sap BHO cannabis concentra...,Concentrates,0.705469,464.756,bhokingz420,United States,United States,2014-06-02 00:00:00
7,"++++1,5 GRAMS+++[4-MEC VERY BIG CRISTALS HIGHE...",Ecstasy,0.138931,91.5264,chemicals_spain,Spain,Worldwide except Australia,2014-06-02 00:00:00
8,1 gr Refined Afghan Opium. High Quality Opium....,Alcohol,0.061098,40.2508,number_five,Undeclared,Worldwide except Australia,2014-06-02 00:00:00
9,1 gram of High Quality Cocaine,Cocaine,0.167758,110.517,Sukey,Germany,Worldwide,2014-06-02 00:00:00
